**mport libraries used in the analysis.**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import chi2_contingency
from statsmodels.stats.proportion import proportions_ztest

****Load dataset**

In [ ]:
# Load data
df = pd.read_csv("click_rates.csv")
print("Dataset Loaded:")
print(df.head())

**Data cleaning & preparation**
add helper column is_click and aggregate counts per headline.

In [ ]:
df["is_click"] = df["Click"].apply(lambda x: 1 if str(x).lower()=="click" else 0)

summary = df.groupby("Headline").agg(
    clicks = ("Rate", lambda x: x[df["Click"]=="Click"].sum()),
    no_clicks = ("Rate", lambda x: x[df["Click"]=="No-click"].sum())
)

summary["total"] = summary["clicks"] + summary["no_clicks"]
summary["CTR"] = summary["clicks"] / summary["total"]

print("\n=== CTR SUMMARY ===")
print(summary)


**Visualize: Bar chart (CTR per Headline)**
plot CTR for each headline.

In [ ]:
plt.figure(figsize=(8,5))
sns.barplot(x=summary.index, y=summary["CTR"], palette="Blues")
plt.title("Click-Through Rate (CTR) by Headline")
plt.ylabel("CTR")
plt.xlabel("Headline")
plt.ylim(0, summary["CTR"].max() * 1.3)
for i, v in enumerate(summary["CTR"]):
    plt.text(i, v+0.005, f"{v:.3f}", ha="center")
plt.show()

**Statistical test: Chi-square test (Are click rates different across headlines)**
use chi-square on the contingency table of clicks/no-clicks.

In [ ]:
chi_data = summary[["clicks", "no_clicks"]].values
chi2, p, dof, expected = chi2_contingency(chi_data)
print("\n=== CHI-SQUARE TEST ===")
print(f"Chi-square = {chi2:.4f}")
print(f"P-value = {p:.6f}")
print("Conclusion:", "SIGNIFICANT difference" if p<0.05 else "No significant difference")

**pairwise A/B tests: Proportion Z-tests**
pairwise compare headlines using two-sample z-test for proportions.

In [ ]:
print("\n=== PROPORTION Z-TESTS (A/B Tests) ===")
headlines = summary.index.tolist()
for i in range(len(headlines)):
    for j in range(i+1, len(headlines)):
        A = headlines[i]; B = headlines[j]
        clicks = [int(summary.loc[A,"clicks"]), int(summary.loc[B,"clicks"])]
        totals = [int(summary.loc[A,"total"]), int(summary.loc[B,"total"])]
        z, p_val = proportions_ztest(clicks, totals)
        print(f"\n{A} vs {B}")
        print(f"Z = {z:.4f}, p = {p_val:.6f} →", "SIGNIFICANT" if p_val<0.05 else "Not significant")


**Heatmap: Clicks vs No-clicks**

In [ ]:
plt.figure(figsize=(6,5))
sns.heatmap(summary[["clicks","no_clicks"]], annot=True, fmt="d", cmap="YlGnBu")
plt.title("Clicks vs No-clicks Heatmap")
plt.show()
